# Tag Frequency Analysis

In [ ]:
from bs4 import BeautifulSoup
import sys
import glob
import numpy as np
import pandas as pd


#SCRAPER_OUTPUT = ".\\tables-metacritic\\pages.npy"
#SCRAPER_OUTPUT = ".\\tables-rotten\\pages.npy"
SCRAPER_OUTPUT = ".\\tables-androidword\\pages.npy"

#SITENAME = "Metacritic"
#SITENAME = "RottenPages"
SITENAME = "Androidword"

N_CLUSTERS = 3

In [ ]:
stats = []

#carico la tabella di output dello scraper che ha come colonne (tipo_template , url , path_pagina_scaricata)
pages = np.load(SCRAPER_OUTPUT)

#creo un Dataframe dalla tabella per poterci fare operazioni sopra in maniera agevole con la libreria pandas
#df = pd.DataFrame(data=pages)
#eliminiamo i duplicati di pagine scaricate (c'era un errore nello scraper che gli faceva scaricare stesse pagine più volte; nella versione finale non ci sarà bisogno di questa operazione)
#df = df.drop_duplicates(subset=0, keep="last") 
#prendo solo i valori del dataframe una volta eliminati i duplicati, in pratica ho la stessa tabella di partenza senza duplicati.
#pages = df.values 

#pages[:,[0, 1]] = pages[:,[1, 0]]

#inizializziamo un dataframe DICT vuoto con due colonne soltanto ( url , template); accoglierà su ogni riga una pagina vettorizzata secondo la frequenza dei tag.
DICT = pd.DataFrame(columns = ["url" , "template"]) 

cc=0
#iniziamo a ciclare sulle pagine per fare le nostre analisi
for p in pages:
    #apriamo il file
    with open(p[2], 'r' , encoding="UTF-8") as f:
        #leggiamo il file come una stringa
        webpage = f.read()
        #passiamo la stringa alla libreria per parsare l'html
        soup = BeautifulSoup(webpage)      
        tags_occurr = []
        #contiamo tutti i tag nella pagina
        for tag in soup.findAll():
            tags_occurr.append(tag.name)      
        #creiamo un dataframe dal vettore così ottenuto
        tags = pd.DataFrame(data=tags_occurr)
        #raggruppiamo i tipi uguali di tag per ottenerne la conta per ciascuno
        tags = tags.groupby([0])
        grouped_tags = tags.size()
        #normalizziamo il vettore così ottenuto
        norm = grouped_tags.div(grouped_tags.sum(axis=0), axis=0)        
        #creiamo una nuova riga di zeri nel dataframe iniziale globale
        DICT.loc[cc] = np.zeros(DICT.shape[1])
        #settiamo url e template per la nuova riga
        DICT.loc[cc , "url"] = p[0]
        DICT.loc[cc , "template"] = p[1]
        #cicliamo il vettore delle tag frequencies e se il tag esiste come colonna ne settiamo il valore
        for i in norm.index:
            if i in DICT.columns:
                DICT.loc[cc , i] = norm[i]
        #altrimenti creiamo la nuova colonna con valore di default 0 e settiamo il nuovo valore
            else: 
                DICT.insert(2, i, 0)
                DICT.loc[cc , i] = norm[i]
    cc += 1
    print(cc)
    if cc%250 == 0:
        print(cc)
    #if cc > 150 :
    #   break

In [ ]:
#decommentare una delle due righe sotto se si vuole salvare o caricare una "tag frequency matrix" (il nome che daremo al nostro sistema per clusterizzare)

DICT.to_csv("TagFrequency-"+SITENAME+".csv" , index=False)
#DICT = pd.read_csv('TagFrequency-'+SITENAME+'.csv') 

In [ ]:
DICT

## K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans

#procediamo con il clustering inizializzando un classificatore kmeans a 3 cluster (dobbiamo classificare 3 templates)

kmeans = KMeans(n_clusters=N_CLUSTERS)

#Prendiamo la nostra matrice delle tag frequencies e togliamo le prime due colonne (corrispondono a url e template)
X = DICT.values[:,2:]

#avviamo il clustering e otteniamo un vettore che riporterà su ciascuna riga il valore del corrispondente gruppo (template) previsto
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
y_kmeans

In [ ]:
DICT.values[:,1]

## Analisi dei risultati del clustering

In [ ]:
#confrontiamo la colonna "template" (ha indice 1) della matrice delle frequenze con il vettore ottenuto dal clustering
#in entrambi i casi che ho provato sono stato fortunato perchè mi ha assegnato da solo gli stessi valori(0,1,2), in caso contrario devono essere rimappati

success = 0
for i,t in enumerate(y_kmeans):
    if i%250 == 0 :
        print("confronting index : "+str(i))
    #print(str(i)+" : "+str(DICT.values[i,1])+" confronting with "+str(t))
    if int(DICT.values[i,1]) == int(t) :
        success += 1
precision = success/len(y_kmeans)
print("Accuracy : "+str(precision))

In [ ]:
trueValues = DICT["template"].values
trueValues = trueValues.reshape((trueValues.size, 1))
y_kmeans = y_kmeans.reshape((y_kmeans.size, 1))
results = np.append(trueValues, y_kmeans, axis=1)
resultsDF = pd.DataFrame(data=results)
resultsDF[0] = pd.to_numeric(resultsDF[0])

In [ ]:
resultsTable = []
for c in range(0 , N_CLUSTERS):
    rilevati = resultsDF.loc[resultsDF[1] == c][0].size
    rilevanti = resultsDF.loc[ (resultsDF[0] == c)][0].size    
    rilevanti_rilevati = resultsDF.loc[(resultsDF[0] == c) & (resultsDF[1] == c)][0].size
    P = rilevanti_rilevati/rilevati
    R = rilevanti_rilevati/rilevanti
    resultsTable.append([P , R])
    print("Precision T"+str(c)+" : "+ str(P))
    print("Recall T"+str(c)+" : "+ str(R))
    print("")

In [ ]:
rtArray = np.array(resultsTable)    
np.save(".\\results\\"+SITENAME+".npy" , rtArray)